In [1]:
#import needed libraries
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import json
import psycopg2
import os

In [2]:
def get_connection():
    conn = {}
    with open('../db_connection/db_config.json') as my_file:
        db_conf = json.load(my_file)
    conn_str = "host={} dbname={} user={} password={}".format(
                db_conf['host'], db_conf['database'], 
                db_conf['user'], db_conf['passw'])
    try:
        conn = psycopg2.connect(conn_str)
        return conn
    except:
        return 'error in connecting to database'

In [3]:
get_connection()

'error in connecting to database'

In [4]:
#get a specific county
def get_county_code_by_id(county_id):
    conn = get_connection()
    all_counties = pd.DataFrame()
    query = "SELECT * FROM common_county WHERE id = '%s' ;" %(county_id)    
    for chunk in pd.read_sql(query, con=conn, chunksize=10):
        all_counties = all_counties.append(chunk)    
    
    return all_counties.head(1)

In [5]:
a = get_county_code_by_id('866cc286-8201-493e-a4db-a9b2a5f64f8e')
# a['name']
# a.columns
a

ImportError: Using URI string without sqlalchemy installed.

In [ ]:
get_county_code_by_id('fd86ddba-a455-431d-9eae-e78bcac4360a')

In [ ]:
def get_county_constituency_codes(county_id):
    '''return codes for constituencies in county'''
    conn = get_connection()
    all_constituencies = DataFrame()
    query = "SELECT * FROM common_constituency WHERE county_id = '%s' ;" %(county_id)   
    for chunk in pd.read_sql(query, con=conn, chunksize=1000):
        all_constituencies = all_constituencies.append(chunk)        
    return all_constituencies

In [ ]:
get_county_constituency_codes('866cc286-8201-493e-a4db-a9b2a5f64f8e')

In [ ]:
#get all the wards in a constituency
def get_constituency_wards(constituency_id):
    '''return ids for wards in a constituency'''
    conn = get_connection()
    all_wards = DataFrame()
    
    query = "SELECT * FROM common_ward WHERE constituency_id = '%s' ;" %(constituency_id)
    for chunk in pd.read_sql(query, con=conn, chunksize=100):
        all_wards = all_wards.append(chunk)
    

    all_wards = all_wards[['name','id','constituency_id']]

    return all_wards

In [ ]:
get_constituency_wards('66eababf-7185-4479-919a-3448838d2f4f')

In [ ]:
def get_ward_facilities(ward_id):
    '''return ids for facilities in a ward'''
    conn = get_connection()
    all_facilities = DataFrame()
    
    query = "SELECT * FROM facilities_facility WHERE ward_id = '%s' ;" %(ward_id)
    for chunk in pd.read_sql(query, con=conn, chunksize=100):
        all_facilities = all_facilities.append(chunk)   

#     all_facilities = all_facilities[['name','id','constituency_id']]

    return all_facilities

In [ ]:
get_ward_facilities('63a51953-aedb-45fb-add3-1c17afdae544')

In [ ]:
#get all the facilities in a county
def get_county_facilities(county_id):
    conn = get_connection()
    county = get_county_code_by_id(county_id)
    
    #get constituencies in the county
    constituencies = get_county_constituency_codes(county_id)
    constituencies = constituencies.rename(index=str, columns={'id':'constituency_id', 'name': "constituency_name"})
    
    #get all the wards for the county
    all_wards = DataFrame()
    for index, constituency in constituencies.iterrows():
        chunk = get_constituency_wards(constituency['constituency_id'])        
        all_wards = all_wards.append(chunk)    
    
    all_facilities = DataFrame()
    for index, ward in all_wards.iterrows():
        all_facilities = all_facilities.append(get_ward_facilities(ward['id']))
        
    return all_facilities

In [ ]:
county_facilities = get_county_facilities('866cc286-8201-493e-a4db-a9b2a5f64f8e')
county_facilities.columns

In [ ]:
def get_constituency_by_id(constituency_id):
    conn = get_connection()
    all_constituencies = pd.DataFrame()
    query = "SELECT * FROM common_constituency WHERE id = '%s' ;" %(constituency_id)   
    for chunk in pd.read_sql(query, con=conn, chunksize=100):
        all_constituencies = all_constituencies.append(chunk)

    # all_constituencies = all_constituencies[all_constituencies['id']==constituency_id]
    # import pdb
    # pdb.set_trace()
    
    return all_constituencies.head(1)

In [ ]:
def get_ward_facilities_filter(ward_id, filters=None):
    '''return ids for facilities in a ward'''
    conn = get_connection()
    all_facilities = DataFrame()
    
    query = "SELECT * FROM facilities_facility WHERE ward_id = '%s' ;" %(ward_id)
    for chunk in pd.read_sql(query, con=conn, chunksize=100):
        all_facilities = all_facilities.append(chunk)   


    if filters:
        for key, value in filters.iteritems():            
            all_facilities = all_facilities[all_facilities[key] == value]

    return all_facilities

In [ ]:
filters={
    "facility_type_id": "6511a5d0-eaca-47fe-b953-2ba5831e21b5",
    "keph_level_id": "7b8cced5-f9a1-4f18-aa2e-f2241ef9c984",    
}

In [ ]:
filters

In [ ]:
get_ward_facilities_filter('63a51953-aedb-45fb-add3-1c17afdae544', filters)

In [ ]:
ward_facilities = get_ward_facilities_filter('63a51953-aedb-45fb-add3-1c17afdae544', filters)
ward_facilities

In [ ]:
def get_ward_by_id(ward_id, in_json=False):
    conn = get_connection()
    all_wards = pd.DataFrame()
    query = "SELECT * FROM common_ward WHERE id = '%s' ;" %(ward_id)
    all_wards = df = pd.read_sql(query, con=conn)
    if in_json:
        return all_wards.head(1).to_json(orient='records')
    else:
        return all_wards.head(1)

In [ ]:
ward = get_ward_by_id('e5b6dea5-d8de-4480-b853-2379b90e73b1')
ward

In [ ]:
def country_facilities():
    conn = get_connection()
    all_facilities = pd.DataFrame()
    query = '''SELECT  common_county.id AS county_id, COUNT(facilities_facility.id) as number_of_facilities , SUM(number_of_beds) AS number_of_beds, SUM(number_of_cots) AS number_of_cots
        FROM facilities_facility , common_ward , common_constituency , common_county 
        WHERE facilities_facility.ward_id = common_ward.id 
            AND common_ward.constituency_id = common_constituency.id 
            AND common_constituency.county_id = common_county.id
        GROUP BY(common_county.id)'''
    all_facilities = pd.read_sql(query, con=conn)
    return all_facilities

In [ ]:
cf = country_facilities()
cf

In [ ]:
country_facilities()

In [ ]:
c = country_facilities()

In [ ]:
c[c['county_id'] == '866cc286-8201-493e-a4db-a9b2a5f64f8e']

In [ ]:
get_county_code_by_id('866cc286-8201-493e-a4db-a9b2a5f64f8e')

In [ ]:
def country_summary():
    conn = get_connection()
    all_facilities = pd.DataFrame()
    query = '''SELECT  common_county.id AS county_id, COUNT(facilities_facility.id) as number_of_facilities , SUM(number_of_beds) AS number_of_beds, SUM(number_of_cots) AS number_of_cots
        FROM facilities_facility , common_ward , common_constituency , common_county 
        WHERE facilities_facility.ward_id = common_ward.id 
            AND common_ward.constituency_id = common_constituency.id 
            AND common_constituency.county_id = common_county.id
        GROUP BY(common_county.id)'''
    all_facilities = pd.read_sql(query, con=conn)
    return all_facilities

In [ ]:
country_summary()

In [ ]:
def country_detailed_summary():
    conn = get_connection()
    all_facilities = pd.DataFrame()
    facility_types_query = '''SELECT facilities_facility.id as count, common_county.name AS county_name, facilities_facilitytype.name  as facility_type_name
        FROM facilities_facility , common_ward , common_constituency , common_county, facilities_facilitytype 
        WHERE facilities_facility.ward_id = common_ward.id 
            AND common_ward.constituency_id = common_constituency.id
            AND facilities_facilitytype.id = facilities_facility.facility_type_id
            AND common_constituency.county_id = common_county.id'''
    all_facilities = pd.read_sql(facility_types_query, con=conn).groupby(['county_name','facility_type_name']).count().unstack().T.fillna(0).xs('count', axis=0, drop_level=True).T
    
    country_summary =[]
    return all_facilities

In [ ]:
countryf = country_detailed_summary()
countryf.to_json(orient='table')

In [ ]:
def country_keph_level_summary():
    conn = get_connection()
    all_facilities = pd.DataFrame()
    facility_types_query = '''SELECT facilities_facility.id as count, common_county.name AS county_name, facilities_kephlevel.name  as keph_level_name
        FROM facilities_facility , common_ward , common_constituency , common_county, facilities_kephlevel
        WHERE facilities_facility.ward_id = common_ward.id 
            AND common_ward.constituency_id = common_constituency.id
            AND facilities_kephlevel.id = facilities_facility.keph_level_id
            AND common_constituency.county_id = common_county.id'''
    all_facilities = pd.read_sql(facility_types_query, con=conn).groupby(['county_name','keph_level_name']).count().unstack().T.fillna(0).xs('count', axis=0, drop_level=True).T
    
    country_summary =[]
    return all_facilities

In [ ]:
country_keph = country_keph_level_summary()

In [ ]:
country_keph.to_json(orient='table')

In [ ]:
def county_keph_level_summary(county_id):
    conn = get_connection()
    constitiency_summary = pd.DataFrame()
    facility_types_query = '''SELECT facilities_facility.id as count, common_constituency.name AS constituency_name, facilities_kephlevel.name  as keph_level_name
        FROM facilities_facility , common_ward , common_constituency , common_county, facilities_kephlevel
        WHERE facilities_facility.ward_id = common_ward.id 
            AND common_ward.constituency_id = common_constituency.id
            AND facilities_kephlevel.id = facilities_facility.keph_level_id
            AND common_constituency.county_id = common_county.id
            AND common_county.id = '%s' ''' %(county_id)
    constitiency_summary = pd.read_sql(facility_types_query, con=conn).groupby(['constituency_name','keph_level_name']).count().unstack().T.fillna(0).xs('count', axis=0, drop_level=True).T
        
    return constitiency_summary

In [ ]:
county_keph_level_summary('866cc286-8201-493e-a4db-a9b2a5f64f8e')

In [ ]:
def get_county_summary(county_id,in_json=False):
    '''return a summary of beds cots and facilities for all counties'''
    conn = get_connection()
    country_summary = pd.DataFrame()
    query = '''SELECT  common_constituency.name AS constituency_name, COUNT(facilities_facility.id) as number_of_facilities , SUM(number_of_beds) AS number_of_beds, SUM(number_of_cots) AS number_of_cots
        FROM facilities_facility , common_ward , common_constituency , common_county 
        WHERE facilities_facility.ward_id = common_ward.id 
            AND common_ward.constituency_id = common_constituency.id
            AND common_constituency.county_id = common_county.id
            AND common_county.id = '%s'
        GROUP BY(common_constituency.name)'''%(county_id)
    country_summary = pd.read_sql(query, con=conn)
    

    if in_json:
        return country_summary.to_json(orient='records')
    else:
        return country_summary

In [ ]:
get_county_summary('866cc286-8201-493e-a4db-a9b2a5f64f8e', False)

In [ ]:
def county_facility_type_summary(county_id):
    conn = get_connection()
    constitiency_summary = pd.DataFrame()
    facility_types_query = '''SELECT facilities_facility.id as count, common_constituency.name AS constituency_name, facilities_facilitytype.name  as facility_type_name
        FROM facilities_facility , common_ward , common_constituency , common_county, facilities_facilitytype
        WHERE facilities_facility.ward_id = common_ward.id 
            AND common_ward.constituency_id = common_constituency.id
            AND facilities_facilitytype.id = facilities_facility.facility_type_id
            AND common_constituency.county_id = common_county.id
            AND common_county.id = '%s' ''' %(county_id)
    constitiency_summary = pd.read_sql(facility_types_query, con=conn).groupby(['constituency_name','facility_type_name']).count().unstack().T.fillna(0).xs('count', axis=0, drop_level=True).T
        
    return constitiency_summary

In [ ]:
def country_beds_summary():
    conn = get_connection()
    country_summary = pd.DataFrame()
    facility_types_query = '''SELECT facilities_facility.number_of_beds as total, common_county.name AS county_name
        FROM facilities_facility , common_ward , common_constituency , common_county
        WHERE facilities_facility.ward_id = common_ward.id 
            AND common_ward.constituency_id = common_constituency.id            
            AND common_constituency.county_id = common_county.id'''
    country_summary = pd.read_sql(facility_types_query, con=conn).groupby(['county_name']).sum().unstack().T.fillna(0).xs('total', axis=0, drop_level=True).T
    return country_summary

In [ ]:
country_beds_summary().to_json()

In [ ]:
def country_cots_summary():
    conn = get_connection()
    country_summary = pd.DataFrame()
    facility_types_query = '''SELECT facilities_facility.number_of_cots as total, common_county.name AS county_name
        FROM facilities_facility , common_ward , common_constituency , common_county
        WHERE facilities_facility.ward_id = common_ward.id 
            AND common_ward.constituency_id = common_constituency.id            
            AND common_constituency.county_id = common_county.id'''
    country_summary = pd.read_sql(facility_types_query, con=conn).groupby(['county_name']).sum().unstack().T.fillna(0).xs('total', axis=0, drop_level=True).T
    return country_summary

In [ ]:
country_cots_summary()

In [ ]:
def county_detailed_summary(county_id):
    conn = get_connection()
    all_facilities = pd.DataFrame()
    facility_types_query = '''SELECT facilities_facility.id as count, common_constituency.name AS constituency_name, facilities_facilitytype.name  as facility_type_name
        FROM facilities_facility , common_ward , common_constituency , common_county, facilities_facilitytype 
        WHERE facilities_facility.ward_id = common_ward.id 
            AND common_ward.constituency_id = common_constituency.id
            AND facilities_facilitytype.id = facilities_facility.facility_type_id
            AND common_constituency.county_id = common_county.id
            AND common_county.id = '%s' '''%(county_id)
    all_facilities = pd.read_sql(facility_types_query, con=conn).groupby(['constituency_name','facility_type_name']).count().unstack().T.fillna(0).xs('count', axis=0, drop_level=True).T
    
    country_summary =[]
    return all_facilities

In [ ]:
county_detailed_summary('d69bfd25-18f1-4640-8e02-40f97df76a1a')